# Running ALS on MovieLens (pySpark)

[ALS](https://spark.apache.org/docs/latest/api/python/_modules/pyspark/ml/recommendation.html#ALS) (Alternating Least Squares) is a well-known collaborative filtering algorithm.

This notebook provides an example of how to utilize and evaluate ALS pySpark ML (DataFrame-based API) implementation, meant for large-scale distributed datasets. We use a smaller dataset in this example to run ALS efficiently on Data Science Virtual Machine.

In [1]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")
import os
import numpy as np
from zipfile import ZipFile
import papermill as pm
import pyspark
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType

from reco_utils.dataset.url_utils import maybe_download
from reco_utils.dataset.spark_splitters import spark_random_split
from reco_utils.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation


print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))


System version: 3.6.0 | packaged by conda-forge | (default, Feb  9 2017, 14:36:55) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
Spark version: 2.3.1


Set the default parameters.

In [2]:
# top k items to recommend
TOP_K = 10

# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

### 0. Set up Spark context

The following settings work well for debugging locally on VM - change when running on a cluster. We set up a giant single executor with many threads and specify memory cap. 

In [3]:
# the following settings work well for debugging locally on VM - change when running on a cluster
# set up a giant single executor with many threads and specify memory cap
spark = SparkSession \
    .builder \
    .appName("ALS pySpark") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g")\
    .config("spark.executor.cores", "32")\
    .config("spark.executor.memory", "8g")\
    .config("spark.memory.fraction", "0.9")\
    .config("spark.memory.stageFraction", "0.3")\
    .config("spark.executor.instances", 1)\
    .config("spark.executor.heartbeatInterval", "36000s")\
    .config("spark.network.timeout", "10000000s")\
    .config("spark.driver.maxResultSize", "50g")\
    .getOrCreate()


### 1. Download the MovieLens dataset

In [4]:
# MovieLens data have different data-format for each size of dataset
data_header = False
if MOVIELENS_DATA_SIZE == "100k":
    separator = "\t"
    data_name = "u.data"
    data_folder = "ml-100k"
elif MOVIELENS_DATA_SIZE == "1m":
    separator = "::"
    data_name = "ratings.dat"
    data_folder = "ml-1m"
elif MOVIELENS_DATA_SIZE == "10m":
    separator = "::"
    data_name = "ratings.dat"
    data_folder = "ml-10M100K"
elif MOVIELENS_DATA_SIZE == "20m":
    separator = ","
    data_name = "ratings.csv"
    data_folder = "ml-20m"
    data_header = True
else:
    raise ValueError("Invalid data size. Should be one of {100k, 1m, 10m, or 20m}")

# Download dataset zip file and decompress if haven't done yet
data_path = os.path.join(data_folder, data_name)
if not os.path.exists(data_path):
    filename = "ml-" + MOVIELENS_DATA_SIZE + ".zip"
    filepath = maybe_download(
        "http://files.grouplens.org/datasets/movielens/" + filename, filename
    )

    with ZipFile(filepath, "r") as zf:
        zf.extractall()

    # remove zip file we already used
    os.remove(filepath)


In [5]:
# Note: The DataFrame-based API for ALS currently only supports integers for user and item ids.
schema = StructType(
    (
        StructField("UserId", IntegerType()),
        StructField("MovieId", IntegerType()),
        StructField("Rating", FloatType()),
        StructField("Timestamp", IntegerType()),
    )
)

# pySpark's read csv currently doesn't support multi-character delimiter, thus we manually handle that
if len(separator) > 1:
    raw_data = spark.sparkContext.textFile(data_path) \
        .map(lambda l: l.split(separator)).map(lambda s: ','.join(map(str, s)))
    data = spark.read.csv(raw_data, schema=schema, sep=',', header=data_header)
else:
    data = spark.read.csv(data_path, schema=schema, sep=separator, header=data_header)

data.show()

+------+-------+------+---------+
|UserId|MovieId|Rating|Timestamp|
+------+-------+------+---------+
|   196|    242|   3.0|881250949|
|   186|    302|   3.0|891717742|
|    22|    377|   1.0|878887116|
|   244|     51|   2.0|880606923|
|   166|    346|   1.0|886397596|
|   298|    474|   4.0|884182806|
|   115|    265|   2.0|881171488|
|   253|    465|   5.0|891628467|
|   305|    451|   3.0|886324817|
|     6|     86|   3.0|883603013|
|    62|    257|   2.0|879372434|
|   286|   1014|   5.0|879781125|
|   200|    222|   5.0|876042340|
|   210|     40|   3.0|891035994|
|   224|     29|   3.0|888104457|
|   303|    785|   3.0|879485318|
|   122|    387|   5.0|879270459|
|   194|    274|   2.0|879539794|
|   291|   1042|   4.0|874834944|
|   234|   1184|   2.0|892079237|
+------+-------+------+---------+
only showing top 20 rows



### 2. Split the data using the Spark random splitter provided in utilities

In [6]:
train, test = spark_random_split(data, ratio=0.75, seed=123)
print ("N train", train.count())
print ("N test", test.count())

N train 75193
N test 24807


### 3. Train the ALS model on the training data, and get the top-k recommendations for our testing data

To predict movie ratings, we use the rating data in the training set as users' explicit feedbacks.

When our goal is to recommend top k movies a user is likely to watch, on the other hand, we utilize the ratings as implicit feedbacks.

In [7]:
header = {
    "userCol": "UserId",
    "itemCol": "MovieId",
    "ratingCol": "Rating",
}


# implicitPrefs=True for recommendation, False for rating prediction
als = ALS(
    rank=40,
    maxIter=15,
    implicitPrefs=True,
    alpha=0.1,
    regParam=0.01,
    coldStartStrategy='drop',
    nonnegative=True,
    **header
)

In [8]:
model = als.fit(train)


In [9]:
recommendations = model.recommendForUserSubset(test, TOP_K)
recommendations.show()
    

+------+--------------------+
|UserId|     recommendations|
+------+--------------------+
|   471|[[71, 0.57152903]...|
|   463|[[285, 1.1577708]...|
|   833|[[185, 1.1997313]...|
|   496|[[204, 0.974352],...|
|   148|[[181, 0.9232193]...|
|   540|[[100, 1.0301069]...|
|   392|[[302, 1.0355105]...|
|   243|[[275, 0.8969344]...|
|   623|[[181, 1.0735292]...|
|   737|[[181, 0.47953993...|
|   897|[[69, 1.0488142],...|
|   858|[[286, 1.1212317]...|
|    31|[[268, 0.78563535...|
|   516|[[286, 0.6772065]...|
|   580|[[405, 1.0718408]...|
|   251|[[121, 1.228331],...|
|   451|[[259, 1.2970893]...|
|    85|[[170, 1.2421695]...|
|   137|[[405, 1.103399],...|
|   808|[[288, 0.8915927]...|
+------+--------------------+
only showing top 20 rows



In [10]:
# Convert to reco util's ranking evaluator format
top_k = recommendations.select('UserId', F.explode('recommendations').alias('r')) \
    .select('UserId', 'r.*')
top_k.show()

+------+-------+----------+
|UserId|MovieId|    rating|
+------+-------+----------+
|   471|     71|0.57152903|
|   471|    588|0.55487996|
|   471|    432|0.55319667|
|   471|    418|0.54289067|
|   471|    501| 0.5107881|
|   471|     99| 0.4807951|
|   471|    419|0.44345152|
|   471|     95|0.44070157|
|   471|     91|0.43045348|
|   471|    143|0.41670722|
|   463|    285| 1.1577708|
|   463|    124| 1.0746053|
|   463|    286| 1.0422602|
|   463|    116| 0.9322226|
|   463|     14|0.91807663|
|   463|    302| 0.9128054|
|   463|    269|0.88212854|
|   463|      1| 0.8625988|
|   463|    283| 0.8478566|
|   463|     13|0.84707767|
+------+-------+----------+
only showing top 20 rows



### 4. Evaluate how well ALS performs

In [11]:
test.show()

+------+-------+------+---------+
|UserId|MovieId|Rating|Timestamp|
+------+-------+------+---------+
|     1|      2|   3.0|876893171|
|     1|      3|   4.0|878542960|
|     1|      4|   3.0|876893119|
|     1|      9|   5.0|878543541|
|     1|     11|   2.0|875072262|
|     1|     17|   3.0|875073198|
|     1|     25|   4.0|875071805|
|     1|     28|   4.0|875072173|
|     1|     30|   3.0|878542515|
|     1|     33|   4.0|878542699|
|     1|     43|   4.0|878542869|
|     1|     48|   5.0|875072520|
|     1|     49|   3.0|878542478|
|     1|     52|   4.0|875072205|
|     1|     59|   5.0|876892817|
|     1|     62|   3.0|878542282|
|     1|     65|   4.0|875072125|
|     1|     66|   4.0|878543030|
|     1|     71|   3.0|876892425|
|     1|     78|   1.0|878543176|
+------+-------+------+---------+
only showing top 20 rows



In [12]:
rank_eval = SparkRankingEvaluation(test, top_k, k = TOP_K, col_user="UserId", col_item="MovieId", 
                                    col_rating="Rating", col_prediction="rating", 
                                    relevancy_method="top_k")

In [13]:
print("Model:\tALS",
      "Top K:\t%d" % rank_eval.k,
      "MAP:\t%f" % rank_eval.map_at_k(),
      "NDCG:\t%f" % rank_eval.ndcg_at_k(),
      "Precision@K:\t%f" % rank_eval.precision_at_k(),
      "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n')

Model:	ALS
Top K:	10
MAP:	0.022188
NDCG:	0.091370
Precision@K:	0.090127
Recall@K:	0.073033


### 5. Evaluate rating prediction

In [14]:
als_prediction = ALS(
    rank=40,
    maxIter=15,
    implicitPrefs=False,
    regParam=0.01,
    coldStartStrategy='drop',
    nonnegative=True,
    **header
)

model_prediction = als_prediction.fit(train)

prediction = model_prediction.transform(test)
prediction.show()


+------+-------+------+---------+----------+
|UserId|MovieId|Rating|Timestamp|prediction|
+------+-------+------+---------+----------+
|   406|    148|   3.0|879540276| 3.0810654|
|    27|    148|   3.0|891543129|  2.908636|
|   606|    148|   3.0|878150506|  3.621529|
|   916|    148|   2.0|880843892| 2.1849582|
|   236|    148|   4.0|890117028|  4.351115|
|   602|    148|   4.0|888638517|  4.241503|
|   663|    148|   4.0|889492989|   4.01702|
|   372|    148|   5.0|876869915| 4.4350476|
|   190|    148|   4.0|891033742|   4.93678|
|     1|    148|   2.0|875240799| 2.7891634|
|   297|    148|   3.0|875239619|  3.849859|
|   178|    148|   4.0|882824325| 3.9544783|
|   308|    148|   3.0|887740788| 2.6153567|
|   923|    148|   4.0|880387474|  4.268126|
|    54|    148|   3.0|880937490| 4.2782016|
|   430|    148|   2.0|877226047|  3.016966|
|    92|    148|   2.0|877383934| 2.1105278|
|   447|    148|   4.0|878854729|  4.211315|
|   374|    148|   4.0|880392992|  4.876349|
|   891|  

In [15]:
rating_eval = SparkRatingEvaluation(test, prediction, col_user="UserId", col_item="MovieId", 
                                    col_rating="Rating", col_prediction="prediction")

print("Model:\tALS rating prediction",
      "RMSE:\t%.2f" % rating_eval.rmse(),
      "MAE:\t%f" % rating_eval.mae(),
      "Explained variance:\t%f" % rating_eval.exp_var(),
      "R squared:\t%f" % rating_eval.rsquared(), sep='\n')

Model:	ALS rating prediction
RMSE:	1.12
MAE:	0.869609
Explained variance:	0.005666
R squared:	0.002667


In [16]:
# Record results with papermill for tests
pm.record("map", rank_eval.map_at_k())
pm.record("ndcg", rank_eval.ndcg_at_k())
pm.record("precision", rank_eval.precision_at_k())
pm.record("recall", rank_eval.recall_at_k())

pm.record("rmse", rating_eval.rmse())
pm.record("mae", rating_eval.mae())
pm.record("exp_var", rating_eval.exp_var())
pm.record("rsquared", rating_eval.rsquared())
